In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [73.0 kB]
Hit:13 http://ppa.launchpad.net/cran/

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2021-11-18 07:01:43--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.31MB/s    in 0.7s    

2021-11-18 07:01:45 (1.31 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("GiftCard").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [5]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Gift_Card_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Gift_Card_v1_00.tsv.gz"), sep="\t", header=True,inferSchema=True)

# Show DataFrame
df.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   24371595|R27ZP1F1CD0C3Y|B004LLIL5A|     346014806|Amazon eGift Card...|       Gift Card|          5|            0|          0|   N|                Y|          Five Stars|Great birthday gi...| 2015-08-31|
|         US|   42489718| RJ7RSBCHUDNNE|B004LLIKVU|     473048287|Amazon.com eGift ...|       Gift Card|          5|    

In [6]:
print(df.count())
df = df.dropna()
print(df.count())
df = df.dropDuplicates()
print(df.count())

149086
149080
149080


In [7]:
df.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: string (nullable = true)



In [8]:
# Review id table 
review_id_tabledf=df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_tabledf.show(5)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RR8P8H4144RUD|   28726648|B007V6EVY2|     910961751| 2015-08-31|
|R1QGEWKB99QW3G|   16943853|B00EPLSMGY|     725850175| 2015-08-31|
|R1E6DFR0YQVPCB|   13097243|B00BWDH3VS|     473048287| 2015-08-28|
| RFDF7ZIKH9PVL|   46281218|B00BWDH368|     473048287| 2015-08-25|
| RWGW40MZS35WQ|   29640138|B0066AZGJI|     480731329| 2015-08-22|
+--------------+-----------+----------+--------------+-----------+
only showing top 5 rows



In [9]:
from pyspark.sql.types import *

In [10]:
review_id_tabledf.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: string (nullable = true)



In [11]:
review_id_tabledf = review_id_tabledf.withColumn("customer_id",review_id_tabledf["customer_id"].cast(IntegerType()))\
    .withColumn("product_parent",review_id_tabledf["product_parent"].cast(IntegerType()))\
    .withColumn("review_date",review_id_tabledf["review_date"].cast(DateType()))

In [12]:
review_id_tabledf.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [13]:
print(review_id_tabledf.count())

149080


In [14]:
review_id_tabledf = review_id_tabledf.dropDuplicates(["review_id"])

In [15]:
# product table 
products_tabledf=df.select(["product_id", "product_title"])
products_tabledf.show(5)

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B007V6EVY2|Amazon Gift Card ...|
|B00EPLSMGY|Amazon eGift Card...|
|B00BWDH3VS|Amazon.com eGift ...|
|B00BWDH368|Amazon.com eGift ...|
|B0066AZGJI|Amazon eGift Card...|
+----------+--------------------+
only showing top 5 rows



In [16]:
products_tabledf.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [17]:
# customer table 
customer_tabledf=df.select(["customer_id"])
customer_tabledf.show(5)

+-----------+
|customer_id|
+-----------+
|   28726648|
|   16943853|
|   13097243|
|   46281218|
|   29640138|
+-----------+
only showing top 5 rows



In [18]:
customer_tabledf = df.groupBy('customer_id').agg({"customer_id": "count"})
customer_tabledf.show(5)

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   35594973|                 1|
|   18961565|                 1|
|   45120109|                 1|
|   22720037|                 1|
|   30734075|                 1|
+-----------+------------------+
only showing top 5 rows



In [19]:
customer_tabledf = customer_tabledf.withColumnRenamed("count(customer_id)", "customer_count")
customer_tabledf.show(5)

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   35594973|             1|
|   18961565|             1|
|   45120109|             1|
|   22720037|             1|
|   30734075|             1|
+-----------+--------------+
only showing top 5 rows



In [20]:
customer_tabledf.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [21]:
customer_tabledf = customer_tabledf.withColumn("customer_id",customer_tabledf["customer_id"].cast(IntegerType()))\
    .withColumn("customer_count",customer_tabledf["customer_count"].cast(IntegerType()))

In [22]:
customer_tabledf.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: integer (nullable = false)



In [23]:
# vine table
vine_tabledf=df.select(["review_id", "star_rating","helpful_votes","total_votes","vine"])
vine_tabledf.show(5)

+--------------+-----------+-------------+-----------+----+
|     review_id|star_rating|helpful_votes|total_votes|vine|
+--------------+-----------+-------------+-----------+----+
| RR8P8H4144RUD|          5|            0|          0|   N|
|R1QGEWKB99QW3G|          5|            0|          0|   N|
|R1E6DFR0YQVPCB|          5|            0|          0|   N|
| RFDF7ZIKH9PVL|          5|            0|          0|   N|
| RWGW40MZS35WQ|          5|            0|          0|   N|
+--------------+-----------+-------------+-----------+----+
only showing top 5 rows



In [24]:
vine_tabledf.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



In [25]:
vine_tabledf = vine_tabledf.withColumn("star_rating",vine_tabledf["star_rating"].cast(IntegerType()))\
    .withColumn("helpful_votes",vine_tabledf["helpful_votes"].cast(IntegerType()))\
    .withColumn("total_votes",vine_tabledf["total_votes"].cast(IntegerType()))

In [26]:
vine_tabledf.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)



Postgres Setup

In [27]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://database-1-instance-1.cnfb0rdvmbzb.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "", 
          "driver":"org.postgresql.Driver"}

In [ ]:
# Write DataFrame to active_user table in RDS
review_id_tabledf.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [ ]:
products_tabledf.write.jdbc(url=jdbc_url, table='products', mode=mode, properties=config)

In [ ]:
customer_tabledf.write.jdbc(url=jdbc_url, table='customers', mode=mode, properties=config)

In [ ]:
vine_tabledf.write.jdbc(url=jdbc_url, table="vine_table", mode=mode, properties=config)